In [128]:
from PIL import Image
import numpy as np
import pickle

In [129]:
im = Image.open('./nosync.nosync/slope_md.tif')
slopearray = np.array(im)

In [130]:
def preprocess_numpy_array(coordinates, original):
    return coordinates['is_in'].to_numpy().reshape(original.shape[0], original.shape[1])

In [131]:
import rioxarray

rds = None
try:
    with open('rds.pickle', 'rb') as handle:
        rds = pickle.load(handle)
except:
    rds = rioxarray.open_rasterio('./nosync.nosync/slope_md.tif')
    df = rds.to_dataframe(name="slope")

    from shapely.geometry import Point
    from shapely.geometry.polygon import Polygon
    import shapefile

    polygons = []
    sf = shapefile.Reader("Reference data/slope_deformations_reference_polygons/reference_data_existing_landslides_polygons.shp")
    shapes = sf.shapes()
    rectangles = []

    for shape in shapes:
        polygons.append(Polygon(shape.points))
        rectangles.append(shape.bbox)

    is_landslide = set()
    for i in range(len(polygons)):
        rectangle = rectangles[i]
        low_x, low_y, high_x, high_y = rectangle[0], rectangle[1], rectangle[2], rectangle[3]
        small_df = df.query('y >= @low_y and y <= @high_y and x >= @low_x and x <= @high_x')
        if(len(small_df) > 0):
            for _, y, x in small_df.index:
                if polygons[i].contains(Point(x, y)):
                    is_landslide.add((x, y))

    bools = np.array([1 if (x, y) in is_landslide else 0 for _, y, x in df.index])
    df.insert(loc = 0, column='is_in', value=bools)
    rds = df.reset_index().drop(columns=['band', 'spatial_ref'])

    with open('rds.pickle', 'wb') as handle:
        pickle.dump(rds, handle)

In [132]:
X_test = list()
Y_test = list()

size_of_square = 100
overlap = 30

np_coords: np.ndarray = preprocess_numpy_array(rds, slopearray)
for i in range(0, slopearray.shape[0] - size_of_square,overlap):
    for j in range(0, slopearray.shape[1] - size_of_square, overlap):
        current_array: np.ndarray = slopearray[i:i+size_of_square, j:j+size_of_square].flatten()
        X_test.append(current_array)
        Y_test.append(np.mean(np_coords[i:i+size_of_square, j:j+size_of_square]))

In [133]:
from sklearn.linear_model import LinearRegression

x = np.array(X_test)
y = np.array(Y_test)

model = LinearRegression().fit(x, y)

In [134]:
def get_coordinates(i, j, coordinates, original):
    top_left = coordinates.loc[i * original.shape[1] + j]
    top_right = coordinates.loc[i * original.shape[1] + j + size_of_square]
    bottom_left = coordinates.loc[(i+size_of_square) * original.shape[1] + j]
    bottom_right = coordinates.loc[(i + size_of_square) * original.shape[1] + j + size_of_square]
    return [top_left, top_right, bottom_left, bottom_right]

In [135]:
final_file_name = './nosync.nosync/slope_whole.tif'
whole_slopes = np.array(Image.open(final_file_name))
result = []

rds2 = rioxarray.open_rasterio(final_file_name)
rds2 = rds2.to_dataframe(name="slope").reset_index().drop(columns=['band', 'spatial_ref', 'slope'])

for i in range(0, whole_slopes.shape[0] - size_of_square,overlap):
    for j in range(0, whole_slopes.shape[1] - size_of_square, overlap):
        current_array: np.ndarray = whole_slopes[i:i+size_of_square, j:j+size_of_square].flatten()
        coordinates = get_coordinates(i, j, rds2, whole_slopes)
        coordinates.append(model.predict(np.array([current_array])))
        result.append(coordinates)

result = np.array(result, dtype=object)

/Users/erehulka/opt/anaconda3/lib/python3.9/site-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (128892672 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [136]:
MAGIC_CONSTANT = 0.8
import shapefile
w = shapefile.Writer('write_rec/outputik', shapefile.POLYGON)
polygons_to_draw = []
for record in result:
    score = record[4][0] 
    if score > MAGIC_CONSTANT:
        left_top_x, left_top_y = record[0]['x'], record[0]['y']
        right_top_x, right_top_y = record[1]['x'], record[1]['y']
        left_bottom_x, left_bottom_y = record[2]['x'], record[2]['y']
        right_bottom_x, right_bottom_y = record[3]['x'], record[3]['y']
        polygons_to_draw.append([
            [left_bottom_x, left_bottom_y], [left_top_x, left_top_y],
            [right_top_x, right_top_y], [right_bottom_x, right_bottom_y]
        ])

w.field('anotherec')
w.record('polygon')
w.poly(polygons_to_draw)

w.close()